In [1]:
import CoolProp.CoolProp as CP
import pandas as pd

# Configurar o fluido como água
fluid = 'Water'

# Função para obter propriedades termodinâmicas
def get_properties(P, T=None, h=None, s=None, Q=None):
    props = {}
    if T is not None:
        props['H'] = CP.PropsSI('H', 'T', T + 273.15, 'P', P * 1e3, fluid) / 1000  # kJ/kg
        props['S'] = CP.PropsSI('S', 'T', T + 273.15, 'P', P * 1e3, fluid) / 1000  # kJ/kg·K
    elif h is not None:
        props['T'] = CP.PropsSI('T', 'P', P * 1e3, 'H', h * 1000, fluid) - 273.15  # °C
        props['S'] = CP.PropsSI('S', 'P', P * 1e3, 'H', h * 1000, fluid) / 1000  # kJ/kg·K
    elif s is not None:
        props['H'] = CP.PropsSI('H', 'P', P * 1e3, 'S', s * 1000, fluid) / 1000  # kJ/kg
        props['T'] = CP.PropsSI('T', 'P', P * 1e3, 'S', s * 1000, fluid) - 273.15  # °C
    if Q is not None:
        props['Q'] = Q
    return props

# Inicializar DataFrame para armazenar os resultados
data = {
    'Corrente': [], 'Pressão (kPa)': [], 'Temperatura (°C)': [],
    'Entalpia (kJ/kg)': [], 'Entropia (kJ/kg·K)': [], 'Título (x)': [], 'Vazão Mássica (kg/h)': []
}

# ========== CÁLCULO DAS CORRENTES ==========

# --- Corrente 1: Entrada da Turbina 1 ---
P1 = 15000
T1 = 600
props = get_properties(P1, T=T1)
data['Corrente'].append(1)
data['Pressão (kPa)'].append(P1)
data['Temperatura (°C)'].append(T1)
data['Entalpia (kJ/kg)'].append(round(props['H'], 1))
data['Entropia (kJ/kg·K)'].append(round(props['S'], 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(10000)

# --- Corrente 2: Saída da Turbina 1 (Isentrópica) ---
P2 = 4000
s2 = props['S']  # Expansão isentrópica
props = get_properties(P2, s=s2)
data['Corrente'].append(2)
data['Pressão (kPa)'].append(P2)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(props['H'], 1))
data['Entropia (kJ/kg·K)'].append(round(s2, 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(10000)

# --- Corrente 3: 82% da Corrente 2 (Caldeira 2) ---
m3 = 0.82 * 10000
data['Corrente'].append(3)
data['Pressão (kPa)'].append(P2)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(props['H'], 1))
data['Entropia (kJ/kg·K)'].append(round(s2, 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(m3)

# --- Corrente 4: 18% da Corrente 2 (Trocador de Calor) ---
m4 = 0.18 * 10000
data['Corrente'].append(4)
data['Pressão (kPa)'].append(P2)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(props['H'], 1))
data['Entropia (kJ/kg·K)'].append(round(s2, 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(m4)

# --- Corrente 5: Saída da Caldeira 2 (Reaquecimento) ---
P5 = 4000
T5 = 600
props = get_properties(P5, T=T5)
data['Corrente'].append(5)
data['Pressão (kPa)'].append(P5)
data['Temperatura (°C)'].append(T5)
data['Entalpia (kJ/kg)'].append(round(props['H'], 1))
data['Entropia (kJ/kg·K)'].append(round(props['S'], 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(m3)

# --- Corrente 6: Saída Turbina 2 (1° Estágio) ---
P6 = 500
s6 = props['S']  # Expansão isentrópica
props = get_properties(P6, s=s6)
data['Corrente'].append(6)
data['Pressão (kPa)'].append(P6)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(props['H'], 1))
data['Entropia (kJ/kg·K)'].append(round(s6, 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(m3)

# --- Corrente 7: 13% da Corrente 6 (Regeneração) ---
m7 = 0.13 * m3
data['Corrente'].append(7)
data['Pressão (kPa)'].append(P6)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(props['H'], 1))
data['Entropia (kJ/kg·K)'].append(round(s6, 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(m7)

# --- Corrente 8: Restante da Corrente 6 ---
m8 = m3 - m7
data['Corrente'].append(8)
data['Pressão (kPa)'].append(P6)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(props['H'], 1))
data['Entropia (kJ/kg·K)'].append(round(s6, 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(m8)

# --- Corrente 9: Saída Turbina 2 (2° Estágio) ---
P9 = 10
s9 = s6  # Expansão isentrópica
props = get_properties(P9, s=s9)
data['Corrente'].append(9)
data['Pressão (kPa)'].append(P9)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(props['H'], 1))
data['Entropia (kJ/kg·K)'].append(round(s9, 3))
data['Título (x)'].append(round(CP.PropsSI('Q', 'P', P9*1e3, 'H', props['H']*1000, fluid), 2))
data['Vazão Mássica (kg/h)'].append(m8)

# --- Corrente 10: Saída do Condensador ---
props = get_properties(P9, Q=0)  # Líquido saturado
data['Corrente'].append(10)
data['Pressão (kPa)'].append(P9)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(props['H'], 1))
data['Entropia (kJ/kg·K)'].append(round(props['S'], 3))
data['Título (x)'].append(0.0)
data['Vazão Mássica (kg/h)'].append(m8)

# --- Corrente 11: Saída da Bomba 1 ---
P11 = 500
h11 = props['H'] + (P11 - P9) * 0.1  # Trabalho da bomba (aproximação)
props = get_properties(P11, h=h11)
data['Corrente'].append(11)
data['Pressão (kPa)'].append(P11)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(h11, 1))
data['Entropia (kJ/kg·K)'].append(round(props['S'], 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(m8)

# --- Corrente 12: Mistura (11 + 7) ---
h12 = (m8 * h11 + m7 * data['Entalpia (kJ/kg)'][6]) / (m8 + m7)
props = get_properties(P11, h=h12)
data['Corrente'].append(12)
data['Pressão (kPa)'].append(P11)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(h12, 1))
data['Entropia (kJ/kg·K)'].append(round(props['S'], 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(m8 + m7)

# --- Corrente 13: Saída da Bomba 2 ---
P13 = 15000
h13 = h12 + (P13 - P11) * 0.1  # Trabalho da bomba
props = get_properties(P13, h=h13)
data['Corrente'].append(13)
data['Pressão (kPa)'].append(P13)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(h13, 1))
data['Entropia (kJ/kg·K)'].append(round(props['S'], 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(m8 + m7)

# --- Corrente 14: Saída do Trocador de Calor (Lado Primário) ---
# Calor transferido da Corrente 4 para a Corrente 13
Q_transfer = m4 * (data['Entalpia (kJ/kg)'][3] - data['Entalpia (kJ/kg)'][15])
h14 = h13 + Q_transfer / (m8 + m7)
props = get_properties(P13, h=h14)
data['Corrente'].append(14)
data['Pressão (kPa)'].append(P13)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(h14, 1))
data['Entropia (kJ/kg·K)'].append(round(props['S'], 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(m8 + m7)

# --- Corrente 15: Saída do Trocador de Calor (Lado Secundário) ---
h15 = data['Entalpia (kJ/kg)'][3] - Q_transfer / m4
props = get_properties(P2, h=h15)
data['Corrente'].append(15)
data['Pressão (kPa)'].append(P2)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(h15, 1))
data['Entropia (kJ/kg·K)'].append(round(props['S'], 3))
data['Título (x)'].append(0.0 if h15 < CP.PropsSI('H', 'P', P2*1e3, 'Q', 0, fluid)/1000 else '-')
data['Vazão Mássica (kg/h)'].append(m4)

# --- Corrente 16: Saída da Bomba 3 ---
P16 = 15000
h16 = h15 + (P16 - P2) * 0.1  # Trabalho da bomba
props = get_properties(P16, h=h16)
data['Corrente'].append(16)
data['Pressão (kPa)'].append(P16)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(h16, 1))
data['Entropia (kJ/kg·K)'].append(round(props['S'], 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(m4)

# --- Corrente 17: Mistura (14 + 16) ---
h17 = (data['Entalpia (kJ/kg)'][13] * (m8 + m7) + data['Entalpia (kJ/kg)'][15] * m4) / 10000
props = get_properties(P16, h=h17)
data['Corrente'].append(17)
data['Pressão (kPa)'].append(P16)
data['Temperatura (°C)'].append(round(props['T'], 1))
data['Entalpia (kJ/kg)'].append(round(h17, 1))
data['Entropia (kJ/kg·K)'].append(round(props['S'], 3))
data['Título (x)'].append('-')
data['Vazão Mássica (kg/h)'].append(10000)

# ========== GERAR TABELA FINAL ==========
df = pd.DataFrame(data)
print(df.to_string(index=False))

KeyError: 'T'